# GPT-3.5-Turbo on GSM8K

In [1]:
import openai
import re
import time

import numpy as np

from tqdm import tqdm
from datasets import load_dataset

In [5]:
import os
openai.api_key =os.environ['APIKEY']

KeyError: 'APIKEY'

In [3]:
os.getenv('APIKEY')

In [4]:
gsm8k = load_dataset('gsm8k', 'main')
validation_index = np.load('../gsm8k/lib_prompt/validation_index.npy')
validation_data = gsm8k['train'].select(validation_index)
gsm8k_test = gsm8k['test']

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [5]:
gsm8k['train'][0]['question']

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [6]:
gsm8k_test = gsm8k['test']

In [7]:
prompt_complex = open('../gsm8k/lib_prompt/prompt_hardest.txt').read()

In [8]:
print(prompt_complex)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [26]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_chain,
    wait_fixed
) 

@retry(wait=wait_chain(*[wait_fixed(3) for i in range(3)] +
                       [wait_fixed(5) for i in range(2)] +
                       [wait_fixed(10)]))
def completion_with_backoff(**kwargs):
    return openai.chat.completions.create(**kwargs)

In [27]:
def test_answer(pred_str, ans_str):
    pattern = '\d*\.?\d+'
    pred = re.findall(pattern, pred_str)
    if(len(pred) >= 1):
        # print(pred_str)
        pred = pred[-1]
        gold = re.findall(pattern, ans_str)
        # print(ans_str)
        gold = gold[-1]
        return pred == gold
    else: return False

def parse_pred_ans(filename):
    with open(filename) as fd: lines = fd.readlines()
    am, a = None, None
    num_q, acc = 0, 0
    current_mode = 'none'
    questions = []
    ans_pred = []
    ans_gold = []
    for l in lines:
        if(l.startswith('Q: ')):
            if(am is not None and a is not None):
                questions.append(q)
                ans_pred.append(am)
                ans_gold.append(a)
                if(test_answer(am, a)):
                    acc += 1
            current_mode = 'q'
            q = l
            num_q += 1
        elif(l.startswith('A_model:')):
            current_mode = 'am'
            am = l
        elif(l.startswith('A:')):
            current_mode = 'a'
            a = l
        else:
            if(current_mode == 'q'): q += l
            elif(current_mode == 'am'): am += l
            elif(current_mode == 'a'): a += l
            else:
                raise ValueError(current_mode)
                
    questions.append(q)
    ans_pred.append(am)
    ans_gold.append(a)
    if(test_answer(am, a)):
        acc += 1
    print('num_q %d correct %d ratio %.4f' % (num_q, acc, float(acc / num_q)))
    return questions, ans_pred, ans_gold

def test_finished(ans_model):
    if('answer is' in ans_model): return True
    else: return False

def extract_ans(ans_model):
    ans_model = ans_model.split('\n')
    ans = []
    residual = []
    for li, al in enumerate(ans_model):
        ans.append(al)
        if('answer is' in al):
            break
    residual = list(ans_model[li + 1:])
    ans = '\n'.join(ans)
    residual = '\n'.join(residual)
    return ans, residual

In [28]:
prompt_q = prompt_complex + '\nQuestion: ' + gsm8k_test[1]['question'] + '\n'

In [29]:
print(prompt_q)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [30]:
response = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {"role": "user", "content": prompt_q},
    ]
)

In [32]:
response.choices[0].message.content

'To find the total number of bolts needed for a robe, we first need to determine how many bolts of white fiber are needed.\nGiven that the robe takes 2 bolts of blue fiber and half that much white fiber, the robe will need 2/2 = 1 bolt of white fiber.\nTherefore, to make the robe, it would take 2 bolts of blue fiber and 1 bolt of white fiber, totaling 2 + 1 = 3 bolts in total. \n\nThe total number of bolts needed for the robe is 3.'

In [34]:
print(response.choices[0].message.content)

To find the total number of bolts needed for a robe, we first need to determine how many bolts of white fiber are needed.
Given that the robe takes 2 bolts of blue fiber and half that much white fiber, the robe will need 2/2 = 1 bolt of white fiber.
Therefore, to make the robe, it would take 2 bolts of blue fiber and 1 bolt of white fiber, totaling 2 + 1 = 3 bolts in total. 

The total number of bolts needed for the robe is 3.


# Complex Prompt Random Sampling, Acc 77.1

In [37]:
i = 0
with open('outputs/test_gpt_3.5_turbo_complex.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'  
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ]
            )
        ans_model = response.choices[0].message.content
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        if(i == 20): break

  1%|▏        | 19/1319 [00:59<1:08:24,  3.16s/it]


In [38]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex.txt')

num_q 20 correct 10 ratio 0.5000


# Complex Prompt Greedy Decoding, Acc 78.85

In [35]:
i = 0
with open('outputs/test_gpt_3.5_turbo_complex_temp_0.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'  
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ],
                temperature=0
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1319/1319 [2:04:18<00:00,  5.65s/it]


In [36]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex_temp_0.txt')

num_q 1319 correct 1040 ratio 0.7885


# Baseline Prompt Greedy Decoding, Acc 74.98

In [8]:
prompt_original = open('../gsm8k/lib_prompt/prompt_original.txt').read()

In [12]:
i = 0
with open('outputs/test_gpt_3.5_turbo_original_temp_0.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_original + '\nQuestion: ' + q + '\n'  
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ],
                temperature=0
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1319/1319 [2:36:17<00:00,  7.11s/it]


In [13]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_original_temp_0.txt')

num_q 1319 correct 989 ratio 0.7498


# Baseline Prompt, Dialog In-Context Learning, Acc 76.8

In [24]:
def make_dialog_prompt(prompt):
    messages = []
    messages.append({"role": "system", "content": "Follow the given examples and answer the question."})
    cases = prompt.split("\n\n")
    for c in cases[:-1]:
        question = c.split("\n")[:2]
        messages.append({"role": "user", "content": "\n".join(question)})
        answer = c.split("\n")[2:]
        messages.append({"role": "assistant", "content": "\n".join(answer)})
    messages.append({"role": "user", "content": cases[-1] + "Let's think step by step"})
    return messages

In [30]:
i = 0
with open('outputs/test_gpt_3.5_turbo_original_dialog_icl.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_original + '\nQuestion: ' + q + '\n'
        dialog_prompt = make_dialog_prompt(prompt_q)
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=dialog_prompt,
              temperature=0
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1319/1319 [2:57:13<00:00,  8.06s/it]


In [31]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_original_dialog_icl.txt')

num_q 1319 correct 1013 ratio 0.7680


# Complex Prompt, Dialog In-Context Learning, Acc 

In [34]:
i = 0
with open('outputs/test_gpt_3.5_turbo_complex_dialog_icl.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'
        dialog_prompt = make_dialog_prompt(prompt_q)
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=dialog_prompt,
              temperature=0
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1319/1319 [2:43:01<00:00,  7.42s/it]


In [35]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex_dialog_icl.txt')

num_q 1319 correct 988 ratio 0.7491


# Lungua 1

In [40]:
print(prompt_complex)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [54]:
from llmlingua import PromptCompressor

llm_lingua = PromptCompressor("Qwen/Qwen2-0.5B", device_map="cpu")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [61]:
print(prompt_complex)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [55]:
compressed_prompt = llm_lingua.compress_prompt(prompt_complex, instruction="", question="", target_token=300)

In [57]:
compressed_prompt['compressed_prompt']

": day?\n's step\n Melanie chapters,3 hours total.\n worksheets they.\nAngelo and Melanie to start with planning 12 hours to study, at 4 day, 23.\n, they to include for breaks lunch. Every they to include0 break,2 hours0 minutes.\n3-minute,.\n 3 lunch  for minutes03So Melanie to hours hours breaks hours.\nThey to no 4 hours day, 15 hours hours day5\nThey will to plan to 4 days to for all the they.\nThe is4\n\nQuestion: Mark's step,= points.\n   pointers\n, and as point points in free.\nAll together his team 0+2 points\n's scored double's of2,  points in.\n scored4.\nThey half's.\n's\nThe total is both scores soThe: Bellabees than deck cards. If she25 times more of each, what of have she0?\n's\n she haveIf she has as have will\nella more than,00 cards\n she25 more\n2+0: A4?\n step\n in,, the of in48 the first.\n baskets of \n areThe ofTheThe:mel, the1 if6?\n's6 were3 then I2.\n of is and price12 then we that022\n\n one appleThe: the she0 new a  many girl?\n,y0, Sarah the.\n boxes05\nThe

In [64]:
len(compressed_prompt['compressed_prompt'].replace('\n',' ').split(' '))

217

In [ ]:
i = 0
with open('outputs/test_gpt_3.5_turbo_complex.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n' 
        prompt_c1 = llm_lingua.compress_prompt(prompt_complex, instruction="", question="", target_token=300)
        prompt_c2 = prompt_c1['compressed_prompt']
        # print("QQQ",prompt_q)
        print("CCC\n",prompt_c1)
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_c2},
                ]
            )
        ans_model = response.choices[0].message.content
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        if(i == 20): break

  0%|                                    | 0/1319 [00:00<?, ?it/s]

CCC
 {'compressed_prompt': ": day?\n's step\n Melanie chapters,3 hours total.\n worksheets they.\nAngelo and Melanie to start with planning 12 hours to study, at 4 day, 23.\n, they to include for breaks lunch. Every they to include0 break,2 hours0 minutes.\n3-minute,.\n 3 lunch  for minutes03So Melanie to hours hours breaks hours.\nThey to no 4 hours day, 15 hours hours day5\nThey will to plan to 4 days to for all the they.\nThe is4\n\nQuestion: Mark's step,= points.\n   pointers\n, and as point points in free.\nAll together his team 0+2 points\n's scored double's of2,  points in.\n scored4.\nThey half's.\n's\nThe total is both scores soThe: Bellabees than deck cards. If she25 times more of each, what of have she0?\n's\n she haveIf she has as have will\nella more than,00 cards\n she25 more\n2+0: A4?\n step\n in,, the of in48 the first.\n baskets of \n areThe ofTheThe:mel, the1 if6?\n's6 were3 then I2.\n of is and price12 then we that022\n\n one appleThe: the she0 new a  many girl?\n,y0

  0%|                         | 1/1319 [00:27<10:14:43, 27.98s/it]

CCC
 {'compressed_prompt': ": day?\n's step\n Melanie chapters,3 hours total.\n worksheets they.\nAngelo and Melanie to start with planning 12 hours to study, at 4 day, 23.\n, they to include for breaks lunch. Every they to include0 break,2 hours0 minutes.\n3-minute,.\n 3 lunch  for minutes03So Melanie to hours hours breaks hours.\nThey to no 4 hours day, 15 hours hours day5\nThey will to plan to 4 days to for all the they.\nThe is4\n\nQuestion: Mark's step,= points.\n   pointers\n, and as point points in free.\nAll together his team 0+2 points\n's scored double's of2,  points in.\n scored4.\nThey half's.\n's\nThe total is both scores soThe: Bellabees than deck cards. If she25 times more of each, what of have she0?\n's\n she haveIf she has as have will\nella more than,00 cards\n she25 more\n2+0: A4?\n step\n in,, the of in48 the first.\n baskets of \n areThe ofTheThe:mel, the1 if6?\n's6 were3 then I2.\n of is and price12 then we that022\n\n one appleThe: the she0 new a  many girl?\n,y0

  0%|                         | 2/1319 [01:00<11:18:03, 30.89s/it]

...

CCC
 {'compressed_prompt': ": day?\n's step\n Melanie chapters,3 hours total.\n worksheets they.\nAngelo and Melanie to start with planning 12 hours to study, at 4 day, 23.\n, they to include for breaks lunch. Every they to include0 break,2 hours0 minutes.\n3-minute,.\n 3 lunch  for minutes03So Melanie to hours hours breaks hours.\nThey to no 4 hours day, 15 hours hours day5\nThey will to plan to 4 days to for all the they.\nThe is4\n\nQuestion: Mark's step,= points.\n   pointers\n, and as point points in free.\nAll together his team 0+2 points\n's scored double's of2,  points in.\n scored4.\nThey half's.\n's\nThe total is both scores soThe: Bellabees than deck cards. If she25 times more of each, what of have she0?\n's\n she haveIf she has as have will\nella more than,00 cards\n she25 more\n2+0: A4?\n step\n in,, the of in48 the first.\n baskets of \n areThe ofTheThe:mel, the1 if6?\n's6 were3 then I2.\n of is and price12 then we that022\n\n one appleThe: the she0 new a  many girl?\n,y0

  1%|▏                        | 8/1319 [04:18<12:08:32, 33.34s/it]

CCC
 {'compressed_prompt': ": day?\n's step\n Melanie chapters,3 hours total.\n worksheets they.\nAngelo and Melanie to start with planning 12 hours to study, at 4 day, 23.\n, they to include for breaks lunch. Every they to include0 break,2 hours0 minutes.\n3-minute,.\n 3 lunch  for minutes03So Melanie to hours hours breaks hours.\nThey to no 4 hours day, 15 hours hours day5\nThey will to plan to 4 days to for all the they.\nThe is4\n\nQuestion: Mark's step,= points.\n   pointers\n, and as point points in free.\nAll together his team 0+2 points\n's scored double's of2,  points in.\n scored4.\nThey half's.\n's\nThe total is both scores soThe: Bellabees than deck cards. If she25 times more of each, what of have she0?\n's\n she haveIf she has as have will\nella more than,00 cards\n she25 more\n2+0: A4?\n step\n in,, the of in48 the first.\n baskets of \n areThe ofTheThe:mel, the1 if6?\n's6 were3 then I2.\n of is and price12 then we that022\n\n one appleThe: the she0 new a  many girl?\n,y0

  1%|▏                        | 9/1319 [04:46<11:36:05, 31.88s/it]

CCC
 {'compressed_prompt': ": day?\n's step\n Melanie chapters,3 hours total.\n worksheets they.\nAngelo and Melanie to start with planning 12 hours to study, at 4 day, 23.\n, they to include for breaks lunch. Every they to include0 break,2 hours0 minutes.\n3-minute,.\n 3 lunch  for minutes03So Melanie to hours hours breaks hours.\nThey to no 4 hours day, 15 hours hours day5\nThey will to plan to 4 days to for all the they.\nThe is4\n\nQuestion: Mark's step,= points.\n   pointers\n, and as point points in free.\nAll together his team 0+2 points\n's scored double's of2,  points in.\n scored4.\nThey half's.\n's\nThe total is both scores soThe: Bellabees than deck cards. If she25 times more of each, what of have she0?\n's\n she haveIf she has as have will\nella more than,00 cards\n she25 more\n2+0: A4?\n step\n in,, the of in48 the first.\n baskets of \n areThe ofTheThe:mel, the1 if6?\n's6 were3 then I2.\n of is and price12 then we that022\n\n one appleThe: the she0 new a  many girl?\n,y0

In [60]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex.txt')

num_q 20 correct 11 ratio 0.5500


In [ ]:
i = 0
with open('outputs/test_gpt_3.5_turbo_complex.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'  
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ]
            )
        ans_model = response.choices[0].message.content
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        if(i == 20): break